# Perturbation of Neoclassical Model

Pablo Winant

Our goal here is to compute a linear approximation of solution to the
neoclassical model, close ot the steady-state.

**Warm-up: install the `ForwardDiff` library. Use it to differentiate
the function below. Check the jacobian function.**

Note: the signature of function `f` needs to be fixed first to allow for
dual numbers as arguments.

In [1]:
using Symbolics, ForwardDiff

In [2]:
test = eps(Float64)
println((1+test)-1)
println((1+test/2)-1)

# test = smallest number : a number smaller than test will disappear in computations

2.220446049250313e-16
0.0


In [3]:
@variables a b x

# Dual number :
# Pour redéfinir opérations 
# import Base: *
# import Base: /
# import Base: +
# import Base: -

a + b*x
Symbolics.derivative(a+b*x, x)

b

In [4]:
ForwardDiff.Dual(1.0,1.0)

Dual{Nothing}(1.0,1.0)

In [5]:
#function f(x::Vector{T}) where T <: Number # Is equivalent to below definition
function f(x::Vector{<:Number})
    #works because 'Dual' (number) of ForwardDiff is a Number !
    a = x[1]
    b = x[2]
    x1 = a+b
    x2 = a*exp(b)
    return [x1,x2]
end

f (generic function with 1 method)

In [6]:
# typeof(ForwardDiff.Dual(2.0,1.0,0.0))
ForwardDiff.jacobian(u->f(u),[0.5,1.0])

2×2 Matrix{Float64}:
 1.0      1.0
 2.71828  1.35914

In [7]:
model = (; α=0.3, β=0.96, γ=4.0, δ=0.1, ρ=0.9)

(α = 0.3, β = 0.96, γ = 4.0, δ = 0.1, ρ = 0.9)

In [8]:
function transition(z::Number, k::Number, i::Number, p)

    (; α, β, γ, δ, ρ) = p
    Z = ρ * z
    K = (1-δ)*k + i

    return (;Z,K)
end


function arbitrage(z::Number, k::Number, i::Number, Z::Number, K::Number, I::Number, p)
    
    # α, β, δ, γ = p works but is order sensitive !! Below formulation reads α will correspond to p.α
    (; α, β, γ, δ, ρ) = p

    # Auxiliary variables today
    y = exp(z)*k^α
    c = y-i

    # Auxiliary variables tomorrow
    Y = exp(Z)*K^α
    C = Y-I

    # Residual of Euler equation
    residual = β*(C/c)^(-γ)*( 1-δ+α*exp(Z)*K^(α-1) ) - 1

    return residual
end

arbitrage (generic function with 1 method)

In [9]:
#! /// To modify defined function to also accept different types than previously defined /// 

arbitrage(s::Vector{T}, x::Vector{T}, S::Vector{T}, X::Vector{T}, p) where T<:Number = arbitrage(s[1],s[2],x[1],S[1],S[2],X[1],p)

# For transition, do not return the named tuple but a vector then
transition(s::Vector{T}, x::Vector{T}, p) where T<:Number = [ transition(s[1],s[2],x[1],p)... ]

# t = (3.0, 2.0, 1.0)
# [ x for x in t] to convert tuple into vector or simpler : [t...]

transition (generic function with 2 methods)

In [10]:
function steady_state(p)

    (; α, β, γ, δ, ρ) = p

    # Steady states
    z = 0.0
    k = ( (1/β - (1-δ))/α )^(1/(α-1))
    i = δ*k

    s = [z,k] # vector of states
    x = [i]   # vector of controls
    
    return (;
        s,
        x)
end

steady_state(model)

(s = [0.0, 2.920822149964071], x = [0.29208221499640713])

In [11]:
#? Check that the steady state is correct using the functions representing the model : i.e transition keeps still

ss = steady_state(model)
# transition(ss.s, ss.x, model) - ss.s 
# returns zero good

arbitrage(ss.s, ss.x, ss.s, ss.x, model)
# returns zero good

0.0

In [12]:
@assert maximum(transition(ss.s, ss.x, model) - ss.s) == 0.0

**Define a structure `PerturbedModel` to hold matrices A,B,C,D,E,F.**

In [13]:
struct PerturbedModel
    A::Matrix
    B::Matrix
    C::Matrix
    D::Matrix
    E::Matrix
    F::Matrix
end

**Write a function
`first_order_model(s::Vector, x::Vector, p)::PerturbedModel`, which
returns the first order model, given the steady-state and the
calibration. Suggestion: use `ForwardDiff.jl` library.**

$f(s_t,x_t,s_{t-1},x_{t-1})$

Derivate of :
$ u \rightarrow f(u,\overline{x},\overline{s},\overline{x})$

In [14]:
# We need to loosen the constraint on the arbitrage arguments : 

# brutal 
arbitrage(s, x, S, X, p) = [ arbitrage(s[1],s[2],x[1],S[1],S[2],X[1],p) ]
transition(s, x, p) = [ transition(s[1],s[2],x[1],p)... ]

# More precise, allows for each parameter to be a different type of Number
# arbitrage(s::Vector{<:Number}, x::Vector{<:Number}, S::Vector{<:Number}, X::Vector{<:Number}, p) = arbitrage(s[1],s[2],x[1],S[1],S[2],X[1],p)

transition (generic function with 3 methods)

In [15]:
s = ss.s
x = ss.x

function first_order_model(s,x,parameters)
    A = ForwardDiff.jacobian( u->arbitrage(u,x,s,x,parameters), s)
    B = ForwardDiff.jacobian( u->arbitrage(s,u,s,x,parameters), x)
    C = ForwardDiff.jacobian( u->arbitrage(s,x,u,x,parameters), s)
    D = ForwardDiff.jacobian( u->arbitrage(s,x,s,u,parameters), x)
    E = ForwardDiff.jacobian( u->transition(u,x,parameters), s)
    F = ForwardDiff.jacobian( u->transition(s,u,parameters), x)

    return PerturbedModel(A,B,C,D,E,F)
end

first_order_model(s,x,model)

PerturbedModel([5.074626865671642 0.5212190203776081], [-3.679193085018409;;], [-4.938626865671642 -0.5538125831185546], [3.679193085018409;;], [0.9 0.0; 0.0 0.9], [0.0; 1.0;;])

**Create a NamedTuple to hold the model parameters.**

**Define two functions:** -
`transition(z::Number, k::Number, i::Number, p)::Tuple{Number}` which
returns productivity and capital at date `t+1` as a function of
productivity, capital and investment at date `t` -
`arbitrage(z::Number, k::Number, i::Number, Z::Number, K::Number, I::Number, p)::Number`
which returns the residual of the euler equation (lower case variable
for date t, upper case for date t+1)

**Using multiple dispatch, define two variants of the same functions,
that take vectors as input and output arguments:** -
`arbitrage(s::Vector{T}, x::Vector{T}, S::Vector{T}, X::Vector{T}, p) where T<:Number` -
`transition(s::Vector{T}, x::Vector{T}, p) where T<:Number`

**Write a function `steady_state(p)::Tuple{Vector,Vector}` which
computes the steady-state of the model computed by hand.** It returns
two vectors, one for the states, one for the controls. Check that the
steady-state satisfies the model equations.

The first order system satisfies:
$$\begin{align}A s_t + B x_t + C s_{t+1} + D x_{t+1} & = & 0 \\\\ 
s_{t+1} & = & E s_t + F x_t
 \end{align}$$

**Define a structure `PerturbedModel` to hold matrices A,B,C,D,E,F.**

**Write a function
`first_order_model(s::Vector, x::Vector, p)::PerturbedModel`, which
returns the first order model, given the steady-state and the
calibration. Suggestion: use `ForwardDiff.jl` library.**

**We look for a linear solution $x_t = X s_t$ . Write the matrix
equation which `X` must satisfy. Write a function
`residual(X::Array, M::PerturbedModel)` which computes the residual of
this equation for a given `X`.**

**Write a function `T(X, M::PerturbedModel)` which implements the time
iteration step.**

**Write function
`linear_time_iteration(X_0::Matrix, m::PerturbedModel)::Matrix` which
implements the time iteration algorithm. Apply it to `X0 = rand(1,2)`
and check that the result satisfies the first order model.**

**Define two linear operators
`L_S(U::Union{Vector, Matrix}, X_0::Matrix, m::PerturbedModel)::Matrix`
and `L_T(U::Matrix, X_0::Matrix, m::PerturbedModel)::Matrix` which
implement the derivatives of the simulation and the time-iteration
operator respectively.**

**Implement a function `spectral_radius(f::Function)::Float64` which
implements the power iteration method to compute the biggest eigenvalues
of the two previously defined operators. Check that Blanchard-Kahn
conditions are met.**

**Write a function
`simulate(s0::Vector, X::Matrix, p, T::Int64)::Tuple{Matrix, Matrix}` to
simulate the model over $T$ periods (by using the formula
$\Delta s_t = (E + F X) \Delta s_{t-1}$. Return a matrix for the states
(one line per date) and another matrix for the controls. Bonus: add a
keyword option to compute variables levels or log-deviations. If
possible, return a DataFrame object.**

**Make some nice plots.**